In [1]:
import pandas as pd

orders = pd.read_csv("/content/orders.csv")
print(orders.head())


   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                  restaurant_name  
0               New Foods Chinese  
1  Ruchi Curry House Multicuisine  
2           Spice Kitchen Punjabi  
3          Darbar Kitchen Non-Veg  
4       Royal Eatery South Indian  


In [2]:
users = pd.read_json("/content/users.json")
print(users.head())


   user_id    name       city membership
0        1  User_1    Chennai    Regular
1        2  User_2       Pune       Gold
2        3  User_3  Bangalore       Gold
3        4  User_4  Bangalore    Regular
4        5  User_5       Pune       Gold


In [4]:
import sqlite3
import pandas as pd # Ensure pandas is imported as it's used by read_sql

# Connect to an in-memory SQLite database
conn = sqlite3.connect(':memory:')

# Read the SQL script from the file
with open('/content/restaurants.sql', 'r') as f:
    sql_script = f.read()

# Execute the SQL script to create tables and insert data
conn.executescript(sql_script)

# Now, read data from the in-memory database
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

print(restaurants.head())

   restaurant_id restaurant_name  cuisine  rating
0              1    Restaurant_1  Chinese     4.8
1              2    Restaurant_2   Indian     4.1
2              3    Restaurant_3  Mexican     4.3
3              4    Restaurant_4  Chinese     4.1
4              5    Restaurant_5  Chinese     4.8


In [5]:
orders_users = pd.merge(
    orders,
    users,
    how="left",
    on="user_id"
)



In [6]:
final_df = pd.merge(
    orders_users,
    restaurants,
    how="left",
    on="restaurant_id"
)


In [7]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


In [9]:
final_df['order_date'] = pd.to_datetime(final_df['order_date'], format='%d-%m-%Y')
final_df.groupby(final_df['order_date'].dt.month)['order_id'].count()

,order_id
order_date,
1,831
2,785
3,903
4,812
5,844
6,784
7,859
8,851
9,812


In [10]:
final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,2023-02-18,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,2023-01-18,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,2023-07-15,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,2023-10-04,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,2023-12-25,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [11]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


In [12]:
from google.colab import files
files.download("final_food_delivery_dataset.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
import pandas as pd

df = pd.read_csv("/content/final_food_delivery_dataset.csv")


In [30]:
#City with highest total revenue from Gold members
df[df['membership']=='Gold'] \
.groupby('city')['total_amount'] \
.sum() \
.sort_values(ascending=False)

,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [29]:
#Cuisine with highest average order value
df.groupby('cuisine')['total_amount'] \
.mean() \
.sort_values(ascending=False)


,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [28]:
#Distinct users with total spend > ₹1000
user_spend = df.groupby('user_id')['total_amount'].sum()
user_spend[user_spend > 1000].count()


np.int64(2544)

In [27]:
#Restaurant rating range with highest revenue
df.groupby('rating')['total_amount'].sum()


,total_amount
rating,
3.0,255018.13
3.1,443863.92
3.2,490913.01
3.3,288212.80
3.4,339942.79
3.5,318822.05
3.6,264193.94
3.7,368173.17
3.8,466878.69


In [26]:
#Among Gold members, city with highest average order value
df[df['membership']=='Gold'] \
.groupby('city')['total_amount'] \
.mean() \
.sort_values(ascending=False)

,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


In [25]:
#Cuisine with lowest number of restaurants but high revenue
df.groupby('cuisine').agg(
    restaurants=('restaurant_id','nunique'),
    revenue=('total_amount','sum')
).sort_values('restaurants')


,restaurants,revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [32]:
#Percentage of orders placed by Gold members
gold_pct = (
    df[df['membership']=='Gold'].shape[0] /
    df.shape[0]
) * 100

round(gold_pct)

50

In [42]:
#Restaurant with highest AOV but < 20 orders
restaurant_stats = df.groupby('restaurant_name_y').agg(
    avg_order=('total_amount','mean'),
    order_count=('order_id','count')
)

restaurant_stats[restaurant_stats['order_count'] < 20] \
.sort_values('avg_order', ascending=False)

,avg_order,order_count
restaurant_name_y,,
Restaurant_294,1040.222308,13
Restaurant_262,1029.473333,18
Restaurant_77,1029.180833,12
Restaurant_193,1026.306667,15
Restaurant_7,1002.140625,16
...,...,...
Restaurant_184,621.828947,19
Restaurant_498,596.815556,18
Restaurant_192,589.972857,14


In [44]:
#Combination contributing highest revenue
df.groupby(['membership','cuisine'])['total_amount'] \
.sum() \
.sort_values(ascending=False)

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [45]:
#Quarter with highest total revenue
df['order_date'] = pd.to_datetime(df['order_date'])
df['quarter'] = df['order_date'].dt.to_period('Q')

df.groupby('quarter')['total_amount'] \
.sum() \
.sort_values(ascending=False)


,total_amount
quarter,
2023Q3,2037385.10
2023Q4,2018263.66
2023Q1,1993425.14
2023Q2,1945348.72
2024Q1,17201.50


In [46]:
#1. How many total orders were placed by Gold members?
df[df['membership'] == 'Gold'].shape[0]


4987

In [47]:
#2. Total revenue generated from Hyderabad (rounded)
round(df[df['city'] == 'Hyderabad']['total_amount'].sum())


1889367

In [48]:
#3️⃣ Number of distinct users who placed at least one order
df['user_id'].nunique()


2883

In [49]:
#4️⃣ Average order value for Gold members (rounded to 2 decimals)
round(df[df['membership'] == 'Gold']['total_amount'].mean(), 2)


np.float64(797.15)

In [50]:
#5️⃣ Number of orders for restaurants with rating ≥ 4.5
df[df['rating'] >= 4.5].shape[0]


3374

In [51]:
#6️⃣ Orders placed in the top-revenue city among Gold members
# Find top revenue city among Gold members
top_city = (
    df[df['membership'] == 'Gold']
    .groupby('city')['total_amount']
    .sum()
    .idxmax()
)

# Count orders in that city by Gold members
df[(df['membership'] == 'Gold') & (df['city'] == top_city)].shape[0]


1337